# Обучение модели

In [1]:
import pandas as pd
import numpy as np
import gdown 
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.sparse as sps
import pickle
from tqdm import tqdm
from joblib import Parallel, delayed
from scipy.sparse import hstack, vstack 

## Загрузка данных

In [2]:
# Загрузка матрицы с рейтингами
gdown.download(url="https://drive.google.com/uc?export=download&id=1D2WB0E7FEHrYHpiXMYezo_Gx8UFGg0_H",  
               output="rating_matrix.npz",
               quiet=False)
# Импорт матрицы
rating_matrix = sps.load_npz("rating_matrix.npz")

Downloading...
From: https://drive.google.com/uc?export=download&id=1D2WB0E7FEHrYHpiXMYezo_Gx8UFGg0_H
To: c:\code\rec_sys\hse_mlds_recsys_project\ML\rating_matrix.npz
100%|██████████| 19.5M/19.5M [00:02<00:00, 9.63MB/s]


In [3]:
# Загрузка тестовых данных
gdown.download(url="https://drive.google.com/uc?export=download&id=1Ud6jFto6e7FW5y0LxxwX8afpmTrvA0u-",  
               output="test_transacrion_df.csv",
               quiet=False)
# Загрузка датафрейма с тестовыми данными
test_df = pd.read_csv("test_transacrion_df.csv", index_col=0)

Downloading...
From: https://drive.google.com/uc?export=download&id=1Ud6jFto6e7FW5y0LxxwX8afpmTrvA0u-
To: c:\code\rec_sys\hse_mlds_recsys_project\ML\test_transacrion_df.csv
100%|██████████| 52.4M/52.4M [00:05<00:00, 9.69MB/s]


In [4]:
# Загрузка кодировщика user_id
gdown.download(url="https://drive.google.com/uc?export=download&id=1eodl9OlaYy3NTu9TpbtPLHNurXxeHlhh",  
               output="encoder.pkl",
               quiet=False)
# Загрузка кодировщика
with open("encoder.pkl", "rb") as f:
    encoder = pickle.load(f)

Downloading...
From: https://drive.google.com/uc?export=download&id=1eodl9OlaYy3NTu9TpbtPLHNurXxeHlhh
To: c:\code\rec_sys\hse_mlds_recsys_project\ML\encoder.pkl
100%|██████████| 800k/800k [00:00<00:00, 2.98MB/s]


# Метрика качества

In [5]:
def rel(r, p):
    return list(pd.Series(p).isin(r))

def mapk(relevant, predicted, k: int = 10):
    N = len(predicted)
    ru = np.array(list(map(rel, relevant, predicted))) # маска релевантных треков каждого польз-ля (k, N) 
    p_at_k = np.array([np.mean(ru[:,:i], axis=1) for i in range(1, k+1)]).T # Prescision at i для каждого user i = 1...k (N, k)
    nu = np.array(list(map(len, relevant))) # кол-во релевантных товаров каждого пользователя (N, ) 
    min_nu_k = 1/np.where(nu > k, k, nu) # min(k, nu)^(-1) для каждого user (N, )
    AP_at_k = np.sum(min_nu_k.reshape(N,1) * p_at_k * ru[:, :k], axis=1) # AP_at_k для каждого user (N, )
    MAP_at_k = np.mean(AP_at_k)
    
    return MAP_at_k


## Коллаборативная фильтрация (User2User)


В качестве функции схожести будет использовано две метрики:

1. Корреляция Пирсона $$s(u, v) = \frac{\sum_{i \in I_u \cap I_v} r_{ui}r_{vi}}{\sqrt{\sum_{i \in I_u} r_{ui} ^2}\sqrt{\sum_{i \in I_v} r_{vi}^2}} $$

2. Мера Жаккара

$$ s(u, v) = \frac{|I_u \cap I_v|}{|I_u \cup I_v|} $$


Корреляция Пирсона немного видоизменена, чтобы подходить под текущую задачу.</br> 
квадрат в формуле можно опустить т.к. рейтинг принимает два значения 0 или 1 (квадрат которых 0 или 1)

Во всех формулах 
* $I_u$ - множество продуктов, купленных пользователем $u$.
* $r_{ui}$ - покупал ли пользователь $u$ продукт $i$ (0 или 1).

Множество соседей определим как $$N(u) = \{ v \in U \setminus \{u\} \mid s(u, v) > \alpha\},$$ где $\alpha \, - $ гиперпараметр.



Для агрегации используется следующая формула:
$$
\hat{r}_{ui} = \frac{\sum_{v \in N(u)} s(u, v) r_{vi}}{\sum_{v \in N(u)} |s(u, v)|}
$$

In [6]:
def pearson(rating_matrix, user_vector):
    """
        Функция расчета корреляции Пирсона
    """
    # Числитель
    numerator = rating_matrix.dot(user_vector.T)
    n = rating_matrix.shape[1]
    # Знаменатель
    denominator = np.sqrt(user_vector.dot(np.ones(shape=(n, 1)))) * np.sqrt(rating_matrix.dot(np.ones(shape=(n, 1))))
    ans = numerator / denominator
    return ans.toarray()

In [60]:
# def get_union(rating_matrix, uid: int = 0):
#     unions = []
#     for i in range(rating_matrix.shape[0]):
#         ans = rating_matrix[i] + rating_matrix[uid]
#         union = sum(np.where(ans.toarray()[0] >= 1, 1, 0))
#         unions.append(union)
#     return np.array(unions)


# def jaccard(ratings_matrix, uid: int):
#     # Пересечение 
#     inter = rating_matrix.dot(rating_matrix[uid].T)
#     union = get_union(rating_matrix=rating_matrix, uid=0)
    
#     return inter / union 

In [7]:
class User2User():

    def __init__(self, rating_matrix, similarity_func, encoder):
        assert similarity_func in [pearson]
        self.rating_matrix = rating_matrix
        self.similarity_func = similarity_func
        self.alpha = 0.02
        self.encoder = encoder

    def predict(self, users_to_recommend: list, n_jobs: int = 3, k: int = 10):
        """
            Параллельное вычисление рекомендаций для пользователей
        """
        predictions = Parallel(n_jobs=3)(delayed(self.recommend)(uid=uid, k=k) for uid in users_to_recommend)
        return predictions

    def cold_start(self):
        """
            Функция холодного старта
            возвращает популярные продукты по убываюнию
        """

        return np.argsort(rating_matrix.T.dot(np.ones(shape=(rating_matrix.shape[1], 1))).T)[::-1]

    def recommend(self,uid: int, k: int):
        """
            Расчет рекомендаций
        """

        # Если ранее такого пользователя не было, то применяется холодный старт
        try:
            uid = self.encoder.transform(np.array([uid]))
        except:
            return self.cold_start()

        # Вектор пользователя
        user_vector = self.rating_matrix[uid]
        # Расчет схожести пользователя с остальными пользователями
        sim_vector = self.similarity_func(rating_matrix=self.rating_matrix, user_vector=user_vector)
        # Индексы со значением функции схожести выше порогового значения
        collab_ids = np.where(sim_vector >= self.alpha)[0]
        # Исключается индекс самого объекта
        collab_ids = collab_ids[collab_ids != uid]
        # Расчет рекомендации 
        sum_of_suvs = np.sum(sim_vector[collab_ids])
        suv_matrix = (self.rating_matrix[collab_ids, ].T.dot(sim_vector[collab_ids])).T/sum_of_suvs
        # rui_est = np.sum(suv_matrix, axis=0)

        return np.argsort(suv_matrix[0])[::-1][:k]

In [8]:
model = User2User(rating_matrix=rating_matrix, 
                  similarity_func=pearson, 
                  encoder=encoder)

In [9]:
# Пользователи с их релевантными покупками
relevant = test_df.groupby(['user_id']).agg({'product_id': 'unique'})['product_id']


In [10]:
%%time
# Рекомендации
predict = model.predict(relevant.index.to_list(), n_jobs=3, k=10)

CPU times: total: 15min
Wall time: 10h 59min 28s


In [12]:
# Оценка качества
mapk(relevant=relevant.to_list(), predicted=predict, k=10)

0.06903240662163014